In [147]:
# 3, AR 1 simulation
import numpy as np

def ar1_simulate(rho, errors, y0):

    y = []
    y.append(y0)
    t = len(errors)
    for i in range(1,t):
        y.append(y[-1] * rho + errors[i-1])
    
    y = np.asarray(y)
  
    return y



In [167]:
# Marie code
def ar(rho,errors,y0):
    t = errors.shape[0]
    y = np.zeros(t)
    y[0] = y0

    for i in range(1,t):
        y[i] = rho*y[i-1] + errors[i-1]
    return y


def sub_rv(rets,k):
    n = rets.shape[0]
    total_sum = 0.0
    for j in range(0,n-k+1):
        inside_sum = 0.0
        for i in range(1,k+1):
            inside_sum += rets[j + i -1]
        total_sum += (inside_sum)**2
    rv_ss = ((n/k)/(n-k+1))*total_sum
    return rv_ss

In [153]:
errors = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
y0 = 1
errors

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [155]:
ar(0.9,errors,0)

array([  0.        ,   1.        ,   2.9       ,   5.61      ,
         9.049     ,  13.1441    ,  17.82969   ,  23.046721  ,
        28.7420489 ,  34.86784401,  41.38105961,  48.24295365,
        55.41865828,  62.87679245,  70.58911321,  78.53020189,
        86.6771817 ,  95.00946353, 103.50851718, 112.15766546])

In [156]:
ar1_simulate(0.9,errors,0)

array([  0.        ,   1.        ,   2.9       ,   5.61      ,
         9.049     ,  13.1441    ,  17.82969   ,  23.046721  ,
        28.7420489 ,  34.86784401,  41.38105961,  48.24295365,
        55.41865828,  62.87679245,  70.58911321,  78.53020189,
        86.6771817 ,  95.00946353, 103.50851718, 112.15766546])

In [158]:
# 2 Subsampled RV
import pandas as pd
def subsampled_rv(rets,k):
    n = len(rets)

    # k is the block size:
    r2 = []
    for i in range(k):
        r2.append((rets.iloc[i::k] ** 2).sum())
    
    r2 = pd.Series(r2).sum()
    num = n / k
    denom = n - k + 1
    multiplier = num / denom

    rv = multiplier * r2
    rv = float(rv)

    return rv






In [51]:

prices = pd.read_csv("mfe-formal-work-1-2020-2021.csv.gz", index_col="time",
compression="infer") 
prices.columns = pd.to_datetime(prices.columns)
ln_p = np.log(prices)
returns = (prices.diff()/prices).dropna()

In [169]:
subsampled_rv( returns["2012-01-03"]*100,1)

5.024909504074747

In [418]:
r = returns["2012-01-03"] * 100
r.index = range(len(r))

In [168]:
sub_rv( r*100,1)

5.02490950407477

In [113]:
len(returns["2011-01-03"]*100) - 1 - 1

23398

In [128]:
rv[1]["2012-01-03"]

5.024908264194439

In [107]:
m = 23400
rv = {}

samp = [23400, 11700,  7800,  5850,  4680,  3900,  2925,  2600,  2340,
        1950,  1800,  1560,  1300,  1170,   975,   936,   900,   780,
         650,   600,   585,   520,   468,   450,   390,   360,   325,
         312,   300,   260,   234,   225,   200,   195,   180,   156,
         150,   130,   120,   117,   104,   100,    90,    78,    75,
          72,    65,    60,    52,    50,    45,    40,    39,    36,
          30,    26,    25,    24,    20,    18,    15,    13]

for i in samp:
    step = m // i
    _rv = ((ln_p.iloc[::step].diff().dropna()*100)**2).sum()
    rv[step] = _rv
rv = pd.DataFrame(rv)

In [404]:
def weighted_hs(rets, lam, window, p):
    wT = (1 - lam) / (1 - lam ** window)
    wT = 1

    wt = np.zeros(window)
    wt[window-1] = wT


    for i in range(window-2,0,-1):
        wt[i] = wt[i+1] * lam
    
    n = rets.shape[0]
    var = np.empty(n)
    var.fill(np.nan)
    save = {}

    for i in range(window,n+1):
        modeled_rets = rets[i-window:i]
        modeled_rets.index = range(window)
        modeled_rets = pd.DataFrame([modeled_rets,wt]).T
        modeled_rets.columns = ["rets","weights"]
        modeled_rets = modeled_rets.sort_values("rets")

        cutoff = int(window * p)
        modeled_rets["rets"][cutoff:] = 0

        sums = (modeled_rets["rets"] * modeled_rets["weights"]).sum()

        var[i-1] = sums
        save[i] = modeled_rets
    
    var = pd.Series(var)

    return  var



    
    





In [412]:
r = r[:1000]

In [417]:
w = 252
l = 0.9

a = weighted_hs(r,0.975,50,0.001)


In [416]:
a

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
995   -0.000183
996   -0.000178
997   -0.000174
998   -0.000169
999   -0.000165
Length: 1000, dtype: float64

In [391]:
b

array([   nan,    nan,    nan,    nan, 12.488, 15.927, 19.366, 22.805,
       26.244, 27.683, 27.322, 25.341, 21.902, 18.463, 17.024, 18.385,
       18.266, 16.815, 46.853, 40.575])

In [406]:
r = pd.Series([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
r = pd.Series([1,2,3,4,5,6,7,8,9,8,7,6,5,4,5,7,5,3,35,3])
r.shape

(20,)